In [1]:
from ngsolve import *
from ngsolve import Draw, curl, grad, sqrt, x, y, z
from netgen.occ import *
from netgen.gui import *
from coil_geometry import *
from ngsolve import *
from ngsolve import x, y, z, sqrt
from math import pi

optfile ./ng.opt does not exist - using default values
togl-version : 2
OCC module loaded
loading ngsolve library
NGSolve-6.2.2406
Using Lapack
Including sparse direct solver UMFPACK
Running parallel using 16 thread(s)


In [2]:
nwindings = 6  # Number of windings
wireradius = 0.001  # Radius of the wire
coilradius = 0.01  # Radius of the coil

# Geometrie laden
geo = create_homo_geometry(nwindings, wireradius, coilradius)

In [3]:
Draw(geo)

In [4]:
mesh = Mesh(
    geo.GenerateMesh()
)
mesh.Curve(1) 

 Face 1 / 10 (parameter space projection)
 Face 2 / 10 (parameter space projection)
 Face 3 / 10 (parameter space projection)
 Face 4 / 10 (parameter space projection)
 Face 5 / 10 (parameter space projection)
 Face 6 / 10 (parameter space projection)
 Face 7 / 10 (parameter space projection)
 Face 8 / 10 (parameter space projection)
 Face 9 / 10 (parameter space projection)
 Face 10 / 10 (parameter space projection)
 Delaunay meshing
 Remove Illegal Elements
 Delaunay meshing
 start tetmeshing
 Success !
 Volume Optimization
 Curve elements, order = 1


In [5]:
mesh.GetBoundaries()

('default',
 'default',
 'default',
 'default',
 'outer',
 'outer',
 'outer',
 'outer',
 'outer',
 'outer')

In [6]:
# Konstanten

mu = 4 * pi * 1e-7  # Magnetische Feldkonstante (S.2)
sigma_coil = 5.87e7  # Leitfähigkeit Kupfer (S.11)
sigma_air = 0 # Leitfähigkeit Luft
k_reg = 1e-5  # Regularisierungsterm

# Geometrische Parameter
i_c = 500 
A_c = pi * wireradius**2  

# Radius in xy-Ebene, +1e-30 vermeidet Division durch 0
r = sqrt(x**2 + y**2) + 1e-30
# Stromdichte j_c gemäß Aufgabenstellung
j_c = CoefficientFunction(
    ((nwindings * i_c / A_c) * (y / r), -(nwindings * i_c / A_c) * (x / r), 0)
)

In [7]:
# H(curl)-Raum definieren
order = 2
V = HCurl(
    mesh, order=order, nograds=True, dirichlet="outer"  # Dirichlet-Rand auf "outer"
)
u, v = V.TnT()
gfA = GridFunction(V)


# Bilinearform und Linearform
a = BilinearForm(V, symmetric=True)
# (1/mu) * (curl(u), curl(v))
a += (1 / mu) * InnerProduct(curl(u), curl(v)) * dx
# Regularisierung
a += k_reg * InnerProduct(u, v) * dx


# Preconditioner

# Linearform: Stromdichte

f = LinearForm(V)

# WICHTIG: die Stromdichte nur in Subdomain "coil"
# -> dx("coil")
f += InnerProduct(j_c, v) * dx("coil")

# Assemble
#with TaskManager():
a.Assemble()
f.Assemble()

print(f"Matrix size A: {a.mat.height} x {a.mat.width}")
print(f"Vector size f: {f.vec.size}")
#print(f"Präkonditionierer-Matrixgröße: {pre.mat.height} x {pre.mat.width}")

Matrix size A: 211873 x 211873
Vector size f: 211873


In [8]:
print(f"Matrix size: {a.mat.height} x {a.mat.width}")
print(f"Vector size: {f.vec.size}")

Matrix size: 211873 x 211873
Vector size: 211873


In [8]:
print(mesh.GetMaterials())  # Gibt die Namen der Subdomains ("coil", "air") aus
print(mesh.GetBoundaries())  # Gibt die Namen der Randbedingungen ("outer") aus

('coil', 'air')
('out', 'coilbnd', 'in', 'outer', 'outer', 'outer', 'outer', 'outer', 'outer')


In [9]:
print(f"Linearform size: {f.vec.size}")

Linearform size: 694234


In [9]:
# Lösen
pre = Preconditioner(
    a,
    "multigrid"
)
# Update the preconditioner to initialize it
pre.Update()

print(f"Präkonditionierer-Matrixgröße: {pre.mat.height} x {pre.mat.width}")
inv = CGSolver(
    a.mat, pre.mat, precision=1e-8, maxsteps=200  # statt tol=1e-8
)  # statt maxiter=200
inv.GetSteps()
with TaskManager():
    gfA.vec.data = inv * f.vec


# Visualisierung
Draw(gfA, mesh, "VectorPotential_A")
Draw(curl(gfA), mesh, "MagField_B")

hcurl smoothingblocks, SmoothingType = 2
Präkonditionierer-Matrixgröße: 211873 x 211873
0 nan


In [11]:
# Calculate the residual manually
residual_vec = f.vec - a.mat * gfA.vec
residual_norm = residual_vec.Norm()

# Check if the solver converged
steps = inv.GetSteps()
if steps < 200 and residual_norm < 1e-8:
    print(f"Solver converged in {steps} steps with residual norm {residual_norm}")
else:
    print(f"Solver did not converge: steps={steps}, residual norm={residual_norm}")

# Check GridFunction norm
solution_norm = gfA.vec.Norm()
if solution_norm > 0 and solution_norm < 1e10:  # Arbitrary upper bound for sanity check
    print(f"Solution norm is valid: {solution_norm}")
else:
    print("Solution norm is invalid or zero")

# Check if mesh materials and boundaries are correct
print("Materials in mesh:", mesh.GetMaterials())
print("Boundaries in mesh:", mesh.GetBoundaries())

# Debug matrix and vector sizes
print(f"Matrix size: {a.mat.height} x {a.mat.width}")
print(f"Vector size: {f.vec.size}")

Solver did not converge: steps=1, residual norm=17006.075648232254
Solution norm is invalid or zero
Materials in mesh: ('coil', 'air')
Boundaries in mesh: ('out', 'coilbnd', 'in', 'outer', 'outer', 'outer', 'outer', 'outer', 'outer')
Matrix size: 694234 x 694234
Vector size: 694234
